In [1]:
import gurobipy as gp
from gurobipy import GRB
import sys

# Data Processing
import pandas as pd
import numpy as np
import random

In [2]:
tabla_final = pd.read_csv('Resultados_Boosting_Tree.csv', sep = ",")
tabla_final = tabla_final[["ID_CLIENTE","probabilidad_de_aceptar"]]
print(tabla_final)

        ID_CLIENTE  probabilidad_de_aceptar
0        1375586.0                 0.024083
1        1050612.0                 0.003222
2        1050613.0                 0.002569
3        1050615.0                -0.001019
4        1050616.0                 0.002581
...            ...                      ...
602555    899549.0                 0.035544
602556   1441442.0                 0.027612
602557   1454431.0                -0.000314
602558     56812.0                 0.019501
602559    981034.0                 0.034644

[602560 rows x 2 columns]


In [3]:
array_cliente_probabilidad = tabla_final.to_numpy()
array_cliente_probabilidad = array_cliente_probabilidad.tolist()

In [4]:
# Crear un objeto de modelo de Gurobi
model = gp.Model()

Restricted license - for non-production use only - expires 2024-10-28


In [5]:
# Conjuntos del modelo

I = array_cliente_probabilidad #Conjunto de personas preaprobadas

In [6]:
# Obtenemos las cardinalidades de cada conjunto
car_i = len(I)

In [7]:
# Parámetros del modelo
C = 1000 # Cobertura

p = list() # puntaje de posibilidad de la adquisición de una tarjeta de crédito para la persona preaprobada i.
for i in range(car_i):
    p.append(array_cliente_probabilidad[i][1])
U = 29587 #utilidades generadas por una persona
CPA_base = 8238

In [8]:
# Creamos las variables de decisión

e = model.addVars(car_i, vtype=GRB.BINARY, name="e")  # 1 si a la persona se le envia cualquier tipo de publicidad, 0 en otro caso.

In [15]:
# Seteamos la función objetivo

model.setObjective((gp.quicksum(e[i] * p[i] for i in range(car_i)))/(gp.quicksum(e[i] for i in range(car_i))), sense=GRB.MAXIMIZE)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "src/gurobipy/linexpr.pxi", line 568, in gurobipy.LinExpr.__truediv__
TypeError: float() argument must be a string or a number, not 'gurobipy.LinExpr'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Cristobal/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/4f/dbgwrrz13h1_vh4b2d4htbn80000gn/T/ipykernel_20655/674696661.py", line 3, in <module>
    model.setObjective((gp.quicksum(e[i] * p[i] for i in range(car_i)))/(gp.quicksum(e[i] for i in range(car_i))), sense=GRB.MAXIMIZE)
  File "src/gurobipy/linexpr.pxi", line 570, in gurobipy.LinExpr.__truediv__
gurobipy.GurobiError: Divisor must be a constant

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Cristobal/Library/Python/3.9/lib

In [13]:
# Agregamos las restricciones

# Cobertura fija
model.addConstr(gp.quicksum(e[i] for i in range(car_i)) == C, "c1")
# CPA < Ingreso
model.addConstrs(123.36 * ( (gp.quicksum(e[i] * p[i] for i in range(car_i)))/(gp.quicksum(e[i] for i in range(car_i))))^(-0.994) <= U, "c2")
# Limitar el número de ofertas enviadas a una persona
model.addConstrs(123.36 * ( (gp.quicksum(e[i] * p[i] for i in range(car_i)))/(gp.quicksum(e[i] for i in range(car_i))))^(-0.994) >= CPA_base , "c3")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "src/gurobipy/linexpr.pxi", line 568, in gurobipy.LinExpr.__truediv__
TypeError: float() argument must be a string or a number, not 'gurobipy.LinExpr'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Cristobal/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/4f/dbgwrrz13h1_vh4b2d4htbn80000gn/T/ipykernel_20655/3660822398.py", line 6, in <module>
    model.addConstrs(123.36 * ( (gp.quicksum(e[i] * p[i] for i in range(car_i)))/(gp.quicksum(e[i] for i in range(car_i))))^(-0.994) <= U, "c2")
  File "src/gurobipy/linexpr.pxi", line 570, in gurobipy.LinExpr.__truediv__
gurobipy.GurobiError: Divisor must be a constant

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Cristobal/Library/Pyt

In [14]:
# Optimizamos el modelo

model.optimize()

# Imprimimos la solución óptima

if model.status == GRB.OPTIMAL:
    print("Encontrada solución óptima!")
    print("Valor de función objetivo: ", model.objVal)
    print("Variables:")
    for var in model.getVars():
        print(var.varName, var.x)
else:
    print("No se encontró solución factible.")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [25]:
valores_e = [e[i].x for i in range(car_i)]
df = pd.DataFrame({'Personas_que_se_envia':valores_e})
df

,Personas_que_se_envia
0,1.0
1,-0.0
2,-0.0
3,-0.0
4,-0.0
...,...
602555,1.0
602556,1.0
602557,-0.0
602558,1.0


In [26]:
tabla_con_id_cliente_mas_probabilidad = pd.merge(tabla_final, df, left_index=True, right_index=True)

tabla_con_id_cliente_mas_probabilidad

,ID_CLIENTE,probabilidad_de_aceptar,Personas_que_se_envia
0,1375586.0,0.024083,1.0
1,1050612.0,0.003222,-0.0
2,1050613.0,0.002569,-0.0
3,1050615.0,-0.001019,-0.0
4,1050616.0,0.002581,-0.0
...,...,...,...
602555,899549.0,0.035544,1.0
602556,1441442.0,0.027612,1.0
602557,1454431.0,-0.000314,-0.0
602558,56812.0,0.019501,1.0


In [27]:
valores_positivos = tabla_con_id_cliente_mas_probabilidad[tabla_con_id_cliente_mas_probabilidad['Personas_que_se_envia'] == 1]
valores_positivos

,ID_CLIENTE,probabilidad_de_aceptar,Personas_que_se_envia
0,1375586.0,0.024083,1.0
9,1050624.0,0.030805,1.0
14,1050586.0,0.024081,1.0
21,1050601.0,0.004928,1.0
26,1050669.0,0.030257,1.0
...,...,...,...
602554,882125.0,0.031876,1.0
602555,899549.0,0.035544,1.0
602556,1441442.0,0.027612,1.0
602558,56812.0,0.019501,1.0


In [28]:
media = valores_positivos["probabilidad_de_aceptar"].mean()
desviacion = valores_positivos["probabilidad_de_aceptar"].std()
raiz_tamaño_muestra = np.sqrt(len(valores_positivos["probabilidad_de_aceptar"]))

Intervalo_superior = media + (1.96 * desviacion / raiz_tamaño_muestra)
Intervalo_superior = Intervalo_superior * len(valores_positivos["probabilidad_de_aceptar"]) * 31850 - len(valores_positivos["probabilidad_de_aceptar"]) *128.8
Intervalo_superior


1359766838.568764

In [29]:
Intervalo_inferior = media - (1.96 * desviacion / raiz_tamaño_muestra)
Intervalo_inferior = Intervalo_inferior* len(valores_positivos["probabilidad_de_aceptar"]) * 31850 - len(valores_positivos["probabilidad_de_aceptar"]) *128.8
Intervalo_inferior

1346734399.689927

In [46]:
probabilidades = valores_positivos["probabilidad_de_aceptar"].tolist()
todos_los_casos = []
aceptacion = []

for j in range (100):

    for i in probabilidades:
        instancia = random.random()
        if i > instancia:
            aceptacion.append(31850-128.8)
        else:
            aceptacion.append(-128.8)

    todos_los_casos.append(sum(aceptacion))
todos_los_casos

[1345455526.8114386,
 2701134903.5763226,
 4046940780.296731,
 5399275907.017139,
 6747597933.737548,
 8103786910.457956,
 9449242437.434443,
 10811833264.552809,
 12162703291.671175,
 13513700718.789541,
 14869889695.907907,
 16220536773.026274,
 17577999750.144638,
 18937023377.263004,
 20291906504.38137,
 21634654781.499737,
 22993614708.618103,
 24349517035.73647,
 25708126612.854836,
 27064442989.9732,
 28413911617.091568,
 29765928244.209934,
 31118709271.3283,
 32469101548.446667,
 33826277875.565033,
 35183358651.67593,
 36527699427.20246,
 37880512302.729,
 39230522378.25553,
 40587826103.78207,
 41947773379.3086,
 43296732404.83514,
 44647697980.36167,
 46009906605.88821,
 47368293231.41474,
 48724450356.94128,
 50077486182.46781,
 51436095757.99435,
 52795151233.52088,
 54149174409.04742,
 55503197584.57395,
 56852634360.10049,
 58205479085.62702,
 59552017511.15356,
 60903174186.68009,
 62266338312.20663,
 63620011137.73316,
 64972919563.2597,
 66316464088.78623,
 676722071

In [47]:
print("Mejor Caso:", max(todos_los_casos))
print("Peor Caso:", min(todos_los_casos))

Mejor Caso: 135342503290.63953
Peor Caso: 1345455526.8114386
